In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re

from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder,MinMaxScaler
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,precision_recall_curve

import xgboost as xgbs
import eli5
import seaborn as sns
import matplotlib.pyplot as plt;

In [2]:
prop_train=pd.read_csv(r'C:\Users\Priyanshi\Downloads\python\Property_train.csv')
prop_test=pd.read_csv(r'C:\Users\Priyanshi\Downloads\python\Property_test_share.csv')

In [3]:
prop_train.shape,prop_test.shape

((62035, 31), (10948, 30))

In [4]:
prop_train["Junk"].value_counts()

0    54433
1     7602
Name: Junk, dtype: int64

In [5]:
#pd.set_option('display.expand_frame_repr',False)

In [6]:
#to display all columns
pd.set_option('display.max_columns',999)

In [7]:
prop_test.head()

,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,PRIMEUNIT,Channel,Zip,InsurancePremiumIndex,PlotType,Architecture,PriceIndex3,Region,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2
0,6LLJ,10645.0,9/22/2010,ZCI3,6015,CAT3,A,missing,9568.0,0,12432.0,6986.0,7133.0,missing,Other,27407,728,WHBI,I3Z9,10860.0,B,5911.0,UA42,BLUE,NC,67549,2006,A,4,8311.0
1,QD0H,9891.0,3/12/2010,RRJQ,7625,OTHER,A,missing,8888.0,0,12173.0,5954.0,6544.0,missing,Agent,32219,1334,32T2,4GZ4,10166.0,A,5370.0,BUNS,BLUE,FL,56570,2004,B,6,7123.0
2,I7GR,8426.0,10/12/2009,ZCI3,6900,CAT2,B,missing,7911.0,0,9134.0,7407.0,7339.0,missing,Other,74135,754,WHBI,TP7R,8500.0,A,6862.0,8YTZ,BLACK,OK,51840,2008,A,1,7994.0
3,5KS5,9269.0,5/20/2010,PRN0,6475,CAT1,B,missing,7773.0,0,8500.0,4241.0,6143.0,missing,Other,80229,1373,WHBI,YIK5,7294.0,A,4872.0,FIVQ,GREEN,CO,83300,2005,A,5,5259.0
4,918O,5366.0,1/13/2009,C8A4,3960,CAT1,A,missing,4134.0,0,5211.0,3352.0,4506.0,missing,Direct,28625,2939,WHBI,YIK5,4120.0,A,3365.0,KT8F,BLACK,NC,95035,2003,A,6,4362.0


In [8]:
prop_train.head()

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,PRIMEUNIT,Channel,Zip,InsurancePremiumIndex,PlotType,Architecture,PriceIndex3,Region,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2
0,0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,0,14224.0,9217.0,10387.0,missing,Direct,21075,623,WHBI,I3Z9,13108.0,A,9022.0,6E9G,GREEN,MD,42077,2008,A,2,10692.0
1,1,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,0,4404.0,2958.0,3400.0,missing,Direct,73129,1689,WHBI,AVYP,3695.0,A,2742.0,FIVQ,SILVER,OK,77258,2002,A,7,3615.0
2,0,QWEV,8210.0,6/23/2010,A9RA,5900,CAT1,B,missing,6922.0,0,8952.0,4568.0,4862.0,missing,Other,29697,2351,W62B,1M5X,7730.0,A,3768.0,4K0H,SILVER,SC,94514,2004,A,6,5805.0
3,0,MK2K,7309.0,3/19/2009,RRJQ,7460,OTHER,A,missing,6300.0,0,7460.0,5361.0,6305.0,missing,Agent,28273,1933,WLQ6,DKOO,6290.0,A,5370.0,ZLXY,WHITE,NC,82302,2002,C,7,6444.0
4,0,GK4G,9182.0,3/3/2010,HE5A,6400,OTHER,B,missing,8330.0,0,9846.0,5428.0,5718.0,missing,Other,29697,482,WHBI,KML6,8583.0,A,4803.0,FIVQ,SILVER,SC,49176,2007,B,3,6286.0


In [9]:
prop_train.dtypes

Junk                      int64
InteriorsStyle           object
PriceIndex8              object
ListDate                 object
Material                 object
PriceIndex9               int64
Agency                   object
AreaIncomeType           object
EnvRating                object
PriceIndex7              object
ExpeditedListing          int64
PriceIndex4              object
PriceIndex1              object
PriceIndex6              object
PRIMEUNIT                object
Channel                  object
Zip                       int64
InsurancePremiumIndex     int64
PlotType                 object
Architecture             object
PriceIndex3              object
Region                   object
PriceIndex5              object
SubModel                 object
Facade                   object
State                    object
NormalisedPopulation      int64
BuildYear                 int64
RegionType               object
PropertyAge               int64
PriceIndex2              object
dtype: o

In [10]:
prop_test.dtypes

InteriorsStyle           object
PriceIndex8              object
ListDate                 object
Material                 object
PriceIndex9               int64
Agency                   object
AreaIncomeType           object
EnvRating                object
PriceIndex7              object
ExpeditedListing          int64
PriceIndex4              object
PriceIndex1              object
PriceIndex6              object
PRIMEUNIT                object
Channel                  object
Zip                       int64
InsurancePremiumIndex     int64
PlotType                 object
Architecture             object
PriceIndex3              object
Region                   object
PriceIndex5              object
SubModel                 object
Facade                   object
State                    object
NormalisedPopulation      int64
BuildYear                 int64
RegionType               object
PropertyAge               int64
PriceIndex2              object
dtype: object

In [11]:
prop_train.nunique()

Junk                         2
InteriorsStyle            1036
PriceIndex8              12686
ListDate                   516
Material                   134
PriceIndex9               2019
Agency                       5
AreaIncomeType               4
EnvRating                    3
PriceIndex7              12078
ExpeditedListing             2
PriceIndex4              13006
PriceIndex1              10053
PriceIndex6              10863
PRIMEUNIT                    3
Channel                      3
Zip                        152
InsurancePremiumIndex      277
PlotType                    13
Architecture                33
PriceIndex3              12309
Region                       3
PriceIndex5              10011
SubModel                   831
Facade                      17
State                       37
NormalisedPopulation     36675
BuildYear                   10
RegionType                   5
PropertyAge                 10
PriceIndex2              11010
dtype: int64

In [12]:
#prop_train.isnull().sum() #no nul values

In [13]:
prop_train.InteriorsStyle.value_counts()

K3BB    2012
RCQE    1713
WEGU    1229
2XWO    1156
V3Y1    1101
        ... 
W0PM       1
9WQQ       1
B76T       1
MDZ7       1
O9WT       1
Name: InteriorsStyle, Length: 1036, dtype: int64

In [14]:
round(prop_train.groupby("InteriorsStyle")["Junk"].mean(),1).value_counts()

0.0    347
0.1    270
0.2    188
0.3     80
1.0     54
0.5     46
0.4     31
0.7     10
0.6      8
0.8      2
Name: Junk, dtype: int64

In [15]:
prop_train["PriceIndex9"]

0        8270
1        3890
2        5900
3        7460
4        6400
         ... 
62030    8745
62031    5360
62032    6670
62033    6600
62034    8700
Name: PriceIndex9, Length: 62035, dtype: int64

In [16]:
for col in ["PriceIndex1","PriceIndex2","PriceIndex3","PriceIndex4","PriceIndex5","PriceIndex6","PriceIndex7","PriceIndex8","PriceIndex9"]:
    prop_train[col]=pd.to_numeric(prop_train[col],errors='coerce')
    prop_test[col]=pd.to_numeric(prop_train[col],errors='coerce')

In [17]:
prop_test["PriceIndex4"].dtype

dtype('float64')

In [18]:
prop_train["PriceIndex1"].var(axis=0)

6066959.202013279

In [19]:
prop_train["PriceIndex4"].dtype

dtype('float64')

In [20]:
prop_train["ListDate"].value_counts().sort_values()#drop
#prop_train["BuildYear"] #drop
#convert in datetime format
#pd.to_datetime(prop_train["ListDate"],infer_datetime_format=True)

1/10/2010       1
12/24/2010      1
5/25/2009       2
1/9/2009        2
2/20/2010       2
             ... 
10/13/2010    311
12/8/2010     312
11/3/2010     315
2/25/2009     325
11/23/2010    329
Name: ListDate, Length: 516, dtype: int64

In [21]:

prop_train["Material"].value_counts()

PRN0    11909
C8A4     8657
ZCI3     7938
NYFD     3228
X7IX     2988
        ...  
8ZEX        1
JXAI        1
5NLN        1
0EFS        1
HTTM        1
Name: Material, Length: 134, dtype: int64

In [22]:
round(prop_train.groupby("Material")["Junk"].mean(),1).value_counts()

0.1    50
0.2    33
0.0    17
0.3    14
0.4     7
0.5     6
1.0     5
0.7     2
Name: Junk, dtype: int64

In [23]:
prop_train["Agency"].value_counts() #ohe

CAT1       21545
CAT2       19862
CAT3       10455
OTHER      10168
missing        5
Name: Agency, dtype: int64

In [24]:
prop_train.groupby("AreaIncomeType")["Agency"].value_counts()

AreaIncomeType  Agency 
A               CAT1       10275
                CAT2        8318
                CAT3        7524
                OTHER       4447
                missing        1
B               CAT2       10499
                CAT1       10177
                OTHER       5101
                CAT3        2369
                missing        4
C               CAT2         222
                CAT1         207
                OTHER        116
                CAT3          99
missing         CAT1         886
                CAT2         823
                OTHER        504
                CAT3         463
Name: Agency, dtype: int64

In [25]:
prop_train.groupby("State")["PropertyAge"].median()

State
AL    4.0
AR    6.0
AZ    4.0
CA    4.0
CO    4.0
FL    4.0
GA    4.0
IA    4.0
ID    4.0
IL    4.0
IN    4.0
KY    3.0
LA    4.0
MA    5.5
MD    4.0
MI    3.0
MN    3.0
MO    4.0
MS    3.0
NC    4.0
NE    3.0
NH    2.0
NJ    4.0
NM    4.0
NV    5.0
NY    2.0
OH    4.0
OK    2.0
OR    4.0
PA    4.0
SC    4.0
TN    4.0
TX    4.0
UT    4.0
VA    4.0
WA    4.0
WV    4.0
Name: PropertyAge, dtype: float64

In [26]:
prop_train.groupby("Architecture")["NormalisedPopulation"].mean()

Architecture
1IKD    68775.892857
1M5X    72221.715692
2MTV    70809.000000
4GZ4    62044.012888
5A9R    79089.904159
5NBT    70090.002232
7IYX    70652.286566
8SPA    67407.704198
AVYP    66724.613284
AZHP    75185.157895
BOLF    81512.480000
CR4E    74370.153846
DKOO    77797.056206
FCXM    68301.401709
GHDG    77804.217391
I3Z9    76746.375209
K92H    75789.234443
KML6    55424.597701
M3C0    74608.029289
O8PU    67564.310345
OJBM    86440.000000
OYQM    58700.000000
PQP2    75778.238095
R0QP    74993.411765
RIAX    74426.620238
RZOR    76903.158585
STJH    76718.544304
TP7R    68255.448894
W86R    77180.753086
WWBR    71941.549020
X120    76430.395387
XDZM    71321.257143
YIK5    73381.304042
Name: NormalisedPopulation, dtype: float64

In [27]:
prop_train["PriceIndex8"].max()

41062.0

In [28]:
prop_train["PriceIndex8"].min()

0.0

In [29]:
prop_train["AreaIncomeType"].value_counts() #ohe

A          30565
B          28150
missing     2676
C            644
Name: AreaIncomeType, dtype: int64

In [30]:
prop_train["EnvRating"].value_counts() #ohe

missing    59092
GREEN       2875
RED           68
Name: EnvRating, dtype: int64

In [31]:
prop_train["ExpeditedListing"].value_counts()

0    60504
1     1531
Name: ExpeditedListing, dtype: int64

In [32]:
prop_train["PRIMEUNIT"].value_counts() #ohe

missing    59092
NO          2886
YES           57
Name: PRIMEUNIT, dtype: int64

In [33]:
prop_train["Channel"].value_counts() #ohe

Direct    34816
Other     14951
Agent     12268
Name: Channel, dtype: int64

In [34]:
prop_train["PlotType"].value_counts()

WHBI    26204
W62B     7572
32T2     6839
DIJX     6113
WLQ6     4978
DSSM     2705
DJOP     1903
7RBQ     1648
I00H     1463
V5R0     1230
8FRL      723
LIGY      652
T559        5
Name: PlotType, dtype: int64

In [35]:
round(prop_train.groupby("PlotType")["Junk"].mean(),2)

PlotType
32T2    0.15
7RBQ    0.09
8FRL    0.15
DIJX    0.16
DJOP    0.14
DSSM    0.12
I00H    0.11
LIGY    0.18
T559    0.00
V5R0    0.16
W62B    0.09
WHBI    0.11
WLQ6    0.13
Name: Junk, dtype: float64

In [36]:
prop_train["Architecture"].value_counts()
round(prop_train.groupby("Architecture")["Junk"].mean(),2).value_counts()

0.12    6
0.15    4
0.10    3
0.17    3
0.00    3
0.16    2
0.34    2
0.14    2
0.33    2
0.06    1
0.28    1
0.13    1
0.08    1
0.21    1
0.50    1
Name: Junk, dtype: int64

In [37]:
prop_train["Region"].value_counts() #ohe

A          59869
B           2160
missing        6
Name: Region, dtype: int64

In [38]:
prop_train["SubModel"].value_counts()
round(prop_train.groupby("SubModel")["Junk"].mean(),1).value_counts()

0.0    321
0.1    184
0.2    151
0.3     57
1.0     43
0.5     42
0.4     19
0.6      5
0.7      5
0.8      4
Name: Junk, dtype: int64

In [39]:
prop_train["Facade"].value_counts()
round(prop_train.groupby("Facade")["Junk"].mean(),2).value_counts()

0.13    6
0.12    4
0.11    2
0.14    1
0.08    1
0.15    1
0.00    1
0.25    1
Name: Junk, dtype: int64

In [40]:
prop_train["State"].value_counts()
round(prop_train.groupby("State")["Junk"].mean(),2).value_counts()

0.14    7
0.11    6
0.09    4
0.12    4
0.06    3
0.15    3
0.10    3
0.18    1
0.05    1
0.24    1
0.13    1
0.04    1
0.08    1
0.00    1
Name: Junk, dtype: int64

In [41]:
prop_train["RegionType"].value_counts() #ohe

A          51862
B           6861
C           3135
OTHER        172
missing        5
Name: RegionType, dtype: int64

In [42]:
def mapping_func_1(df,x,y,prefix='prop_'):
    probs=round(df.groupby(x)[y].mean(),1).to_dict()
    mapping_dict=dict()
    for k,v in probs.items():
        mapping_dict[k]=prefix+str(v).replace(".","")
    return mapping_dict    

In [43]:
prop_train=prop_train.assign( 
 InteriorsStyle=prop_train["InteriorsStyle"].map(mapping_func_1(prop_train,"InteriorsStyle","Junk","IntSt_")),
 Material=prop_train["Material"].map(mapping_func_1(prop_train,"Material","Junk","mat_")),
 SubModel=prop_train["SubModel"].map(mapping_func_1(prop_train,"SubModel","Junk","SubM_")))


In [44]:
def mapping_func_2(df,x,y,prefix='prop_'):
    probs=round(df.groupby(x)[y].mean(),2).to_dict()
    mapping_dict=dict()
    for k,v in probs.items():
        mapping_dict[k]=prefix+str(v).replace(".","")
    return mapping_dict    

In [45]:
prop_train=prop_train.assign(
 PlotType=prop_train["PlotType"].map(mapping_func_2(prop_train,"PlotType","Junk","PlotT_")),
 Architecture=prop_train["Architecture"].map(mapping_func_2(prop_train,"Architecture","Junk","Arch_")),
 Facade=prop_train["Facade"].map(mapping_func_2(prop_train,"Facade","Junk","Fac_")),
 State=prop_train["State"].map(mapping_func_2(prop_train,"State","Junk","State_")))

In [46]:
prop_train.nunique()

Junk                         2
InteriorsStyle              10
PriceIndex8              12685
ListDate                   516
Material                     8
PriceIndex9               2019
Agency                       5
AreaIncomeType               4
EnvRating                    3
PriceIndex7              12077
ExpeditedListing             2
PriceIndex4              13005
PriceIndex1              10052
PriceIndex6              10862
PRIMEUNIT                    3
Channel                      3
Zip                        152
InsurancePremiumIndex      277
PlotType                     9
Architecture                15
PriceIndex3              12308
Region                       3
PriceIndex5              10010
SubModel                    10
Facade                       8
State                       14
NormalisedPopulation     36675
BuildYear                   10
RegionType                   5
PropertyAge                 10
PriceIndex2              11009
dtype: int64

In [47]:
prop_train.dtypes

Junk                       int64
InteriorsStyle            object
PriceIndex8              float64
ListDate                  object
Material                  object
PriceIndex9                int64
Agency                    object
AreaIncomeType            object
EnvRating                 object
PriceIndex7              float64
ExpeditedListing           int64
PriceIndex4              float64
PriceIndex1              float64
PriceIndex6              float64
PRIMEUNIT                 object
Channel                   object
Zip                        int64
InsurancePremiumIndex      int64
PlotType                  object
Architecture              object
PriceIndex3              float64
Region                    object
PriceIndex5              float64
SubModel                  object
Facade                    object
State                     object
NormalisedPopulation       int64
BuildYear                  int64
RegionType                object
PropertyAge                int64
PriceIndex

In [48]:
x_train,x_test=train_test_split(prop_train,test_size=.2,random_state=1)

In [49]:
x_train.shape,x_test.shape

((49628, 31), (12407, 31))

In [50]:
x_train1=x_train.drop(["Junk","BuildYear","ListDate"],1)
y_train1=x_train["Junk"]
x_test1=x_test.drop(["Junk","BuildYear","ListDate"],1)
y_test1=x_test["Junk"]

In [51]:
x_train1.shape,y_train1.shape,x_test1.shape,y_test1.shape

((49628, 28), (49628,), (12407, 28), (12407,))

In [52]:
num_cols=x_train1.select_dtypes(np.number).columns

In [53]:
num_cols

Index(['PriceIndex8', 'PriceIndex9', 'PriceIndex7', 'ExpeditedListing',
       'PriceIndex4', 'PriceIndex1', 'PriceIndex6', 'Zip',
       'InsurancePremiumIndex', 'PriceIndex3', 'PriceIndex5',
       'NormalisedPopulation', 'PropertyAge', 'PriceIndex2'],
      dtype='object')

In [54]:
cat_cols=x_train1.select_dtypes(object).columns

In [55]:
cat_cols

Index(['InteriorsStyle', 'Material', 'Agency', 'AreaIncomeType', 'EnvRating',
       'PRIMEUNIT', 'Channel', 'PlotType', 'Architecture', 'Region',
       'SubModel', 'Facade', 'State', 'RegionType'],
      dtype='object')

In [56]:
pipe_num=make_pipeline(SimpleImputer(strategy="median"),StandardScaler())
pipe_cat=make_pipeline(SimpleImputer(strategy="constant",fill_value="Missing"),
                      OneHotEncoder(handle_unknown="ignore"))

In [57]:
ctrans=make_column_transformer((pipe_num,num_cols),(pipe_cat,cat_cols))

In [58]:
ctrans.fit_transform(x_train1)

<49628x114 sparse matrix of type '<class 'numpy.float64'>'
	with 1389584 stored elements in Compressed Sparse Row format>

In [448]:
ctrans.transform(prop_test)

<10948x114 sparse matrix of type '<class 'numpy.float64'>'
	with 229908 stored elements in Compressed Sparse Row format>

## Logistic Regression

In [511]:
logreg=LogisticRegression(
    solver="liblinear",
    penalty="l1",
    random_state=2,
    max_iter=800,
    class_weight="balanced")

In [512]:
logreg

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=800, multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=2, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [513]:
pipe=make_pipeline(ctrans,logreg)

In [514]:
pipe

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

In [515]:
pipe.fit(x_train1,y_train1)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

In [516]:
pipe.predict(x_test1)

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [517]:
pipe.predict(x_train1)

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [518]:
roc_auc_score(y_train1,pipe.predict_proba(x_train1)[:,1])

0.7902405671775095

In [519]:
roc_auc_score(y_test1,pipe.predict_proba(x_test1)[:,1])

0.7834626789942428

In [520]:
final_pred=pipe.predict(prop_test)

In [521]:
final_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [522]:
submission=pd.DataFrame(data=final_pred)

In [525]:
submission[0].value_counts()

1    10610
0      338
Name: 0, dtype: int64

In [526]:
submission=submission.rename({0:'Junk'},axis=1)

In [527]:
submission

,Junk
0,1
1,1
2,1
3,1
4,1
...,...
10943,1
10944,1
10945,1
10946,1


In [528]:
submission=submission.to_csv("Priyanshi_Project_5.csv",index=False)

## GridSearchCV

In [529]:
ctrans.fit_transform(x_train1)

<49628x114 sparse matrix of type '<class 'numpy.float64'>'
	with 1389584 stored elements in Compressed Sparse Row format>

In [530]:
params={}
params["logisticregression__penalty"]=["l1","l2"]
params["logisticregression__C"]=[.1,.01,1,10]

In [531]:
logreg=LogisticRegression(solver="liblinear",random_state=2)
pipe=make_pipeline(ctrans,logreg)

In [532]:
grid=GridSearchCV(pipe,params,cv=5,scoring="accuracy")

In [533]:
grid

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                  

In [534]:
grid.fit(x_train1,y_train1)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                  

In [535]:
roc_auc_score(y_train1,grid.predict_proba(x_train1)[:,1])

0.788313761783282

In [536]:
roc_auc_score(y_test1,grid.predict_proba(x_test1)[:,1])

0.7822707116936237

In [537]:
grid.predict(x_train1)

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [539]:
pred=grid.predict(prop_test)

In [540]:
submiss=pd.DataFrame(data=pred)

In [542]:
submiss[0].value_counts()

0    10431
1      517
Name: 0, dtype: int64

## XGBoost

In [101]:
strings="xgb__"
parameters={'n_estimators':[20,40,80,150,200],
            'max_depth':range(2,10,1),
            'learning_rate':[.1,.01,.05],
            'reg_alpha':[.1,.01,1,10],
            'reg_lambda':[.1,.01,1,10]}


In [102]:
parameters.items()

dict_items([('n_estimators', [20, 40, 80, 150, 200]), ('max_depth', range(2, 10)), ('learning_rate', [0.1, 0.01, 0.05]), ('reg_alpha', [0.1, 0.01, 1, 10]), ('reg_lambda', [0.1, 0.01, 1, 10])])

In [103]:
xgb_params={}
for k,v in parameters.items():
    xgb_params[strings+k]=v

In [104]:
xgb_params

{'xgb__n_estimators': [20, 40, 80, 150, 200],
 'xgb__max_depth': range(2, 10),
 'xgb__learning_rate': [0.1, 0.01, 0.05],
 'xgb__reg_alpha': [0.1, 0.01, 1, 10],
 'xgb__reg_lambda': [0.1, 0.01, 1, 10]}

In [105]:
ctrans.fit_transform(x_train1)

<49628x114 sparse matrix of type '<class 'numpy.float64'>'
	with 1389584 stored elements in Compressed Sparse Row format>

In [106]:
xgb=xgbs.XGBClassifier()
pipe=Pipeline([('columntransfer',ctrans),('xgb',xgb)])

In [107]:
pipe

Pipeline(memory=None,
         steps=[('columntransfer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                              

In [108]:
grid=RandomizedSearchCV(pipe,xgb_params,cv=5,scoring="accuracy")

In [109]:
grid

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('columntransfer',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('pipeline-1',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('simpleimputer',
                                                                                                SimpleImputer(add_indicator=False,
                                                                      

In [110]:
grid.fit(x_train1,y_train1)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('columntransfer',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('pipeline-1',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('simpleimputer',
                                                                                                SimpleImputer(add_indicator=False,
                                                                      

In [111]:
roc_auc_score(y_train1,grid.predict_proba(x_train1)[:,1])

0.8126512789370515

In [112]:
roc_auc_score(y_test1,grid.predict_proba(x_test1)[:,1])

0.7784581128874882

In [113]:
grid.best_params_

{'xgb__reg_lambda': 1,
 'xgb__reg_alpha': 1,
 'xgb__n_estimators': 150,
 'xgb__max_depth': 9,
 'xgb__learning_rate': 0.01}

In [114]:
grid.predict(x_train1)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [116]:
pred=grid.predict(prop_test)

In [117]:
pred

array([0, 1, 0, ..., 0, 0, 1], dtype=int64)

In [118]:
submiss_3=pd.DataFrame(data=pred)

In [121]:
submiss_3

,0
0,0
1,1
2,0
3,1
4,1
...,...
10943,0
10944,0
10945,0
10946,0


In [120]:
submiss_3[0].value_counts()

0    6481
1    4467
Name: 0, dtype: int64

In [123]:
submiss_3=submiss_3.rename({0:'Junk'},axis=1)

In [124]:
submiss_3

,Junk
0,0
1,1
2,0
3,1
4,1
...,...
10943,0
10944,0
10945,0
10946,0


In [125]:
submiss_3=submiss_3.to_csv("Priyanshi_Project_5_2.csv",index=False)